In [1]:
import pandas as pd
import numpy as np
import requests as rq
import json
from nltk.tokenize import word_tokenize as wt 
from nltk.tokenize import sent_tokenize as st
from nltk.corpus import stopwords
import re

In [2]:
# genre_api = rq.get('https://api.themoviedb.org/3/genre/movie/list?api_key=8265bd1679663a7ea12ac168da84d2e8&language=en-US')
# genr = genre_api.text
# genre_data=json.loads(genr)
# # genre_data['genres']
# gen_d = pd.DataFrame.from_dict(genre_data['genres'])
# # gen_d.to_csv("genre_data")

In [3]:
# api = "https://api.themoviedb.org/3/movie/top_rated?api_key=8265bd1679663a7ea12ac168da84d2e8&language=en-US&page=1"
# movie_api = rq.get(api)
# mov = movie_api.text
# mov_data = json.loads(mov)
# mov_data.pop('page')
# print(mov_data['results'])
# mov_df= pd.DataFrame.from_dict(mov_data['results'])
# mov_df.drop(columns=["adult","backdrop_path","id","original_language","popularity","poster_path","release_date","title","video","vote_average","vote_count"], inplace= True)

# for i in range(2,450):
#     api = "https://api.themoviedb.org/3/movie/top_rated?api_key=8265bd1679663a7ea12ac168da84d2e8&language=en-US&page="+str(i)
#     movie_api = rq.get(api)
#     mov = movie_api.text
#     mov_data = json.loads(mov)
#     mov_data.pop('page')
#     # print(mov_data['results'])
#     newdata=pd.DataFrame.from_dict(mov_data['results'])
#     newdata.drop(columns=["adult","backdrop_path","id","original_language","popularity","poster_path","release_date","title","video","vote_average","vote_count"], inplace= True)
#     # print(newdata)
#     mov_df= pd.concat([mov_df,newdata],ignore_index=True)
#     # newdata.info()
# # mov_df.to_csv("mov_data")


In [6]:
# genre_mapping = dict(zip(gen_d['id'],gen_d['name']))
# mov_df['genre'] = mov_df['genre_ids'].apply(lambda ids: [genre_mapping.get(id) for id in ids])
# mov_df.drop(['genre_ids'],inplace=True,axis=1)
# mov_df['genre'] = mov_df['genre'].apply(lambda x: ', '.join(map(str, x)))

# mov_df['genre'] = mov_df['genre'].str.lower()
# mov_df['original_title'] = mov_df['original_title'].str.lower()
# mov_df['overview'] = mov_df['overview'].str.lower()


In [232]:
mov_df = pd.read_csv('mov_data',header=0,index_col=0)

**Removing Non English Character movie name from 'original_title' column**

In [233]:
english_pattern = r'^[a-zA-Z0-9\s!"#$%&\'()*+,-./:;<=>?@[\\]^_{|}~].*$'
mov_df['english_rows'] = mov_df['original_title'].str.contains(english_pattern, na=True)
mov_df[mov_df['english_rows']==False]
mov_df=mov_df[english_rows]
mov_df.reset_index(drop=True,inplace=True)
mov_df.drop(columns=['english_rows'],inplace=True)
mov_df

,original_title,overview,genre
0,the godfather,"spanning the years 1945 to 1955, a chronicle o...","drama, crime"
1,the shawshank redemption,framed in the 1940s for the double murder of h...,"drama, crime"
2,the godfather part ii,in the continuing saga of the corleone crime f...,"drama, crime"
3,the dark knight,batman raises the stakes in his war on crime. ...,"drama, action, crime, thriller"
4,the green mile,a supernatural tale set on death row in a sout...,"fantasy, drama, crime"
...,...,...,...
6275,death note,a young man comes to possess a supernatural no...,"crime, fantasy, horror, thriller"
6276,stan helsing,it's halloween night and video store clerk sta...,comedy
6277,left behind,a small group of survivors are left behind aft...,"thriller, action, science fiction"
6278,date movie,spoof of romantic comedies which focuses on a ...,"comedy, romance"


**Removing special characters from 'overview' 'genre'**

In [234]:
# replace_special_characters = lambda x: x.replace('.', '').replace(',', '').replace("'", '').replace('"', '')
mov_df['overview']=mov_df['overview'].apply(lambda x: str(x).replace('.', '').replace(',', '').replace("'", '').replace('"', '').replace('-',' ').replace('&','and'))
mov_df['genre']=mov_df['genre'].apply(lambda x: str(x).replace('.', '').replace(',', '').replace("'", '').replace('"', ''))

In [ ]:
#both are required to resolve lookup error
'''nltk.download('punkt')
nltk.download('stopwords')'''

**Stop Word removal and Tokenization of overview Column**

In [235]:
def remove_stopwords(text):
    stopWords = set(stopwords.words('english'))
    words = wt(text)
    filtered_words = [word for word in words if word.lower() not in stopWords]
    return ' '.join(filtered_words)

mov_df['overview']=mov_df['overview'].apply(lambda w : remove_stopwords(w))
mov_df['overview']=mov_df['overview'].apply(lambda x: str(x).replace('lt ', 'leutinent ').replace('dr ', 'doctor '))
# mov_df['overview']=mov_df['overview'].apply(lambda x: wt(x))

In [192]:
# r =mov_df['overview'][4]

In [153]:
# Only if required 
# mov_df['genre']=mov_df['genre'].apply(lambda x: wt(x)) 

**Stemming of overview column**

In [224]:
from nltk.stem import PorterStemmer, SnowballStemmer

In [236]:
def pStem(text):
    stemmed_words = [ps.stem(word) for word in wt(text)]
    return ' '.join(stemmed_words)
mov_df['overview']=mov_df['overview'].apply(lambda x: pStem(x))

In [237]:
from nltk.stem import wordnet
from nltk.stem import WordNetLemmatizer
import spacy
nlp = spacy.load("en_core_web_sm")
lemmatizer = WordNetLemmatizer()

In [238]:
def lemmaOv(text):
    lemm = [word.lemma_ for word in nlp(text)]
    return ' '.join(lemm)
mov_df['overview']=mov_df['overview'].apply(lambda x: lemmaOv(x))

In [ ]:
mov_df['overview']=mov_df['overview'].apply(lambda x: wt(x))

In [242]:
mov_df[['overview']]

,overview
0,"[span, year, 1945, 1955, chronicl, fiction, it..."
1,"[frame, 1940, doubl, murder, wife, lover, upst..."
2,"[continu, saga, corleon, crime, famili, young,..."
3,"[batman, rais, stake, war, crime, help, leutin..."
4,"[supernatur, tale, set, death, row, southern, ..."
...,...
6275,"[young, man, come, possess, supernatur, notebo..."
6276,"[halloween, night, video, store, clerk, stan, ..."
6277,"[small, group, survivor, leave, behind, millio..."
6278,"[spoof, romant, comedi, focus, man, crush, par..."
